## Real Reviews
### Optimizing Business Reviews Using NLP

In [1]:
import json
import pandas as pd
import nltk
import codecs
import sklearn.metrics
import pickle
import sklearn.feature_extraction.text
import sklearn.naive_bayes
import sklearn.cross_validation
from scipy.sparse import coo_matrix, vstack, hstack
from langdetect import detect
from yelpapi import YelpAPI
from pprint import pprint
from textblob import TextBlob
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

//anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [91]:
# Read the Pickled DataFrame
df = pd.read_pickle('data/100k.pkl')

In [2]:
# Read the Yelp reviews dataset into a DataFrame
# Using 100,000 records to build model
# Use 'langdetect' module to only include English reviews
#  for our training set
# with codecs.open('data/yelp_academic_dataset_review.json', 'r', encoding='utf-8') as f:
#     d = [json.loads(f.readline().strip()) for x in range(100000) if detect(json.loads(f.readline().strip())['text'])=="en"]

In [3]:
# Create the DataFrame for training
# df = pd.DataFrame(d)
# len(df)

In [10]:
# Pickle the DataFrame
# df.to_pickle('data/100k.pkl')

In [12]:
df

,business_id,cool,date,funny,review_id,stars,text,type,useful,user_id,polarity,subjectivity,wc
0,2aFiy99vNLklCx3T_tGS9A,0,2010-12-29,0,pXbbIgOXvLuTi_SPs1hQEQ,5,After being on the phone with Verizon Wireless...,review,1,bQ7fQq1otn9hKX-gXRsrgA,0.022857,0.505714,98
1,2LfIuF3_sX6uwe-IR-P0jQ,1,2014-07-14,0,GP6YEearUWrzPtQYSF1vVg,5,Highly recommended. Went in yesterday looking ...,review,0,aW3ix1KNZAvoM8q-WghA3Q,0.395000,0.570625,55
2,2LfIuF3_sX6uwe-IR-P0jQ,1,2013-04-28,0,Uf1Ki1yyH_JDKhLvn2e4FQ,5,What a great place! Modern on Melrose has amaz...,review,2,bgl3j8yJcRO-00NkUYsXGQ,0.413766,0.527792,27
3,2LfIuF3_sX6uwe-IR-P0jQ,0,2012-09-18,0,bRvdVt88MJ_YMTlLbjDLxQ,5,This place is a great for those vintage/mid ce...,review,2,GJ7PTY7huYORFKKg3db3Gw,0.330556,0.505556,77
4,2LfIuF3_sX6uwe-IR-P0jQ,0,2015-04-05,0,LkP1l7sZIwOV6IKNLqQp_A,5,"Great items at a good price. Helpful, easy to...",review,0,UU0nHQtHPMAfLidk8tOHTg,0.644444,0.727778,32
5,2LfIuF3_sX6uwe-IR-P0jQ,1,2014-08-23,1,dJguzIfNKtGkKk8ryOPdfQ,5,I'm a local Real Estate agent and have fallen ...,review,1,OvD92wp0-uuFoGLBymwfKQ,0.239062,0.412500,124
6,2LfIuF3_sX6uwe-IR-P0jQ,0,2014-08-05,0,cNA3oGTmmeRvt10vrvlJOQ,5,Walked into the store and stuck up a conversat...,review,1,AziQIgYIAY6uVw1k8sbtTw,0.175000,0.535000,80
7,2LfIuF3_sX6uwe-IR-P0jQ,2,2013-11-17,3,klVqdjXPM2aM8Y0RsDCPoA,1,"Unfortunately, Yelp does not allow negative st...",review,4,wnzfuir72IZFg5RAPOwWCQ,-0.009318,0.428396,335
8,2LfIuF3_sX6uwe-IR-P0jQ,0,2014-08-23,0,k4xighfFxKv5H-Eic28MNQ,5,I stumbled upon Modern on Melrose earlier in t...,review,0,oKZiDLm0D1PIm1-vRtvONA,0.208860,0.471432,297
9,0czfEgv9KAD4VlIa7ANPWQ,1,2009-04-10,2,_a7Zu2ZSEGO4bl2gvu7OtQ,5,"I love Mint, and even though I'm a guy and the...",review,2,jhhHm3Vk9ZlP21WdY_5R0w,0.301190,0.551190,43


## Look Up Business Data Using Yelp API

In [37]:
# How many unique Businesses?
len(df['business_id'].unique())

24817

In [38]:
# Initialize the Yelp API
yelp_id = ""
yelp_secret = ""

with open('data/pvt.csv') as f:
    for line in f:
        l = line.strip().split(',')
        if l[0] == "yelp_id":
            yelp_id = l[1]
        if l[0] == "yelp_secret":
            yelp_secret = l[1]

yelp_api = YelpAPI(yelp_id, yelp_secret)

In [89]:
# Choose a business and store the response from API
bid = df['business_id'].sample(1).iloc[0]
response = yelp_api.business_query(id=bid)
print bid
print response

KuHNBKXBCk3XXNJlZW7jRQ
{u'is_claimed': False, u'rating': 3.5, u'is_closed': True, u'review_count': 3, u'name': u'Half Moon Sports Grill', u'transactions': [], u'url': u'https://www.yelp.com/biz/half-moon-sports-grill-tempe?adjust_creative=fb-xs7dYOAKPmOWg2AKm6Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=fb-xs7dYOAKPmOWg2AKm6Q', u'price': u'$$$', u'photos': [], u'phone': u'+14804569400', u'image_url': u'', u'coordinates': {u'latitude': 33.348099, u'longitude': -111.943092}, u'display_phone': u'(480) 456-9400', u'id': u'half-moon-sports-grill-tempe', u'categories': [{u'alias': u'tradamerican', u'title': u'American (Traditional)'}, {u'alias': u'amusementparks', u'title': u'Amusement Parks'}], u'location': {u'cross_streets': u'', u'city': u'Tempe', u'display_address': [u'325 W Elliot Rd', u'Ste 104', u'Tempe, AZ 85284'], u'country': u'US', u'address2': u'Ste 104', u'address3': u'', u'state': u'AZ', u'address1': u'325 W Elliot Rd', u'zip_code': u'85284'}}


In [76]:
# Variables from 'response' that we will need
city = response['location']['city']
country = response['location']['country']
name = response['name']

In [9]:
print(city)
print(country)
print(name)

Mississauga
CA
Cyclepath Mississauga


## Sentiment Analysis

In [7]:
# Conduct sentiment analysis on reviews
sentiments = [TextBlob(i).sentiment for i in df['text']]

In [8]:
# Add columns for polarity and subjectivity ranking
df['polarity'] = [i[0] for i in sentiments]
df['subjectivity'] = [i[1] for i in sentiments]

In [9]:
# Add column for review wordcount
df['wc'] = [len(x.split()) for x in df['text']]

In [73]:
df[df['polarity']<-.7].iloc[15]

business_id                                2EJeW2tHqNbAIxoaJ4tMbg
cool                                                            0
date                                                   2015-01-31
funny                                                           0
review_id                                  hwn4pc__i3xHgw0wTe3YHA
stars                                                           1
text            Very bad rude stuff shisha overpriced \nU can ...
type                                                       review
useful                                                          0
user_id                                    Rw39nws0uNobEP0Xbkxmtw
polarity                                                -0.736667
subjectivity                                             0.748889
wc                                                             43
Name: 7925, dtype: object

## NLP Model
### Favorability Model

In [18]:
# We will only look at 5 star or 1 start reviews for training the model
# This implies that all 5 star reviews are definitely "favorable",
#  while 1 star reviews are definitely "unfavorable"
df_stars = df[(df['stars'] == 5) | (df['stars'] == 1)][['text', 'stars']]

In [98]:
# How many records in the new DataFrame?
len(df_stars)

47905

In [20]:
# Binarize the star rankings (1 = Favorable, 0 = Unfavorable)
df_stars['stars'] = df_stars['stars'].map(lambda x: 1 if x == 5 else 0)

In [21]:
# Renumber the index
df_stars.index = range(len(df_stars))

In [20]:
# Create our TFIDF Vectorizer which will be used to
#  rank the importance of words for our classifier
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(ngram_range=(1,2), strip_accents='unicode')
vectorizer.fit(df_stars['text'])

In [94]:
# Transform our sample of reviews into a TFIDF vector
reviews_tfidf = vectorizer.transform(df_stars['text'])

In [23]:
# What are the details of our sparse matrix?
reviews_tfidf

<47905x1251033 sparse matrix of type '<type 'numpy.float64'>'
	with 8398041 stored elements in Compressed Sparse Row format>

In [22]:
# Convert tfidf matrix to DataFrame
reviews_tfidf_arr = reviews_tfidf.toarray()
reviews_tfidf_df = pd.DataFrame(reviews_tfidf_arr, columns=vectorizer.get_feature_names())

In [ ]:
reviews_tfidf_df

In [29]:
# Split sample into Train and Test sets for cross validation
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(reviews_tfidf, df_stars['stars'], test_size=0.3, stratify=df_stars['stars'])

In [96]:
%%timeit -n1 -r1
# Random Forests Cross Validaton on F1 Score
forest = RandomForestClassifier()
forest_acc = sklearn.cross_validation.cross_val_score(forest, X_train, y_train, cv=2, scoring='f1')
print(forest_acc)

[ 0.87128713  0.86877828]
1 loop, best of 1: 2.43 s per loop


In [30]:
# Random Forests Train Test Split
forest = RandomForestClassifier()
forest.fit(X_train, y_train)
for_pred = forest.predict(X_test)
print "f1:", sklearn.metrics.f1_score(for_pred, y_test)
print "recall:", sklearn.metrics.recall_score(for_pred, y_test)
print "precision:", sklearn.metrics.precision_score(for_pred, y_test)
print "accuracy:", sklearn.metrics.accuracy_score(for_pred, y_test)

f1: 0.915394761636
recall: 0.877766735929
precision: 0.956393345802


In [102]:
# Random Forests - Dummy Classifier
dummy = [1 for x in range(len(df_stars['stars']))]
for_pred = forest.predict(reviews_tfidf)
print "f1:", sklearn.metrics.f1_score(for_pred, dummy)
print "recall:", sklearn.metrics.recall_score(for_pred, dummy)
print "precision:", sklearn.metrics.precision_score(for_pred, dummy)
print "accuracy:", sklearn.metrics.accuracy_score(for_pred, dummy)

f1: 0.829029222327
recall: 1.0
precision: 0.707984552761
accuracy: 0.707984552761


In [33]:
# Train a Random Forests model on the entire dataset
forest = RandomForestClassifier()
forest.fit(reviews_tfidf, df_stars['stars'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [180]:
len(df_bin['stars'])

106236

In [54]:
%%timeit -n1 -r1
# Multinomial Naive Bayes Cross Validaton on F1 Score
nb = sklearn.naive_bayes.MultinomialNB()
nb_acc = sklearn.cross_validation.cross_val_score(nb, X_train, y_train, cv=2, scoring='f1')
print(nb_acc)

[ 0.82166264  0.82166264]
1 loop, best of 1: 133 ms per loop


In [60]:
# Train a Naive Bayes model on the entire dataset
nb = sklearn.naive_bayes.MultinomialNB()
nb.fit(x, df_bin['stars'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

## Pickle the Favorability Models

In [34]:
# Pickle the Random Forests model
with open('model/forest_100k_eng.pkl', 'wb') as f:
    pickle.dump(forest, f, protocol=pickle.HIGHEST_PROTOCOL)

In [24]:
# Pickle the TFIDF Vectorizer
with open('model/vectorizer_100k_eng.pkl', 'wb') as f:
    pickle.dump(vectorizer, f, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
# Save the TFIDF Vectorizer (joblib)
joblib.dump(vectorizer, 'model/vectorizer_100k_joblib.pkl')

['model/vectorizer_100k_joblib.pkl']

In [92]:
# Load the Random Forests Model
with open('model/forest_100k_eng.pkl', 'rb') as f:
    forest = pickle.load(f)

In [3]:
# Load the TFIDF Vectorizer
with open('model/vectorizer_100k_eng.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

In [6]:
# Load the TFIDF Vectorizer
vectorizer = joblib.load('model/vectorizer_100k_joblib.pkl')

1 loop, best of 1: 29.7 s per loop


## Test the Model

In [58]:
%%timeit -n1 -r1
# Test the model
vec = vectorizer.transform(["I have mixed feelings about this restaurant.  The service was good, but I did not care much for the food.  Overall, I would say it was not great. Not a fan."])
print forest.predict(vec)

[0]
1 loop, best of 1: 127 ms per loop


In [211]:
df_stars[df_stars['text'].str.contains("disappointing")]

,text,stars
74,Posted the following a couple weeks ago:\n\n(f...,1
83,The only reason that I can even begin to imagi...,1
211,I LOVE Jessica and her services! She is by far...,1
238,I have been to other 5 & DIner restaurants and...,0
268,First let me say that I love Mexican food and ...,0
304,"For some reason, even though there were only a...",0
369,"Wow, who cancels on a bride ON THE DAY OF THE ...",0
380,"Very disappointing - boat was too big, too cro...",0
428,Oh man. This is such a disappointing review fo...,0
532,Update: so it turns out after coming home and ...,0


In [224]:
df_stars.iloc[83]['text']

u"The only reason that I can even begin to imagine as to why reviewers say it's nothing special is because a) they were expecting the typical Mexican food. This is Halisco style food - not the same! Think of it as more of a gourmet style Mexican restaurant. And b) they ordered the chicken tacos. Admittedly, they sort of suck. I've been coming here with my family for years! After they moved across town to the new location, we followed.\n\nAmbiance: It's a CUTE restaurant with an old town Mexican theme. Subdued lighting with eccentric and rich coloring.\n\nService: You are evil spawn if you don't like M&M. They work hard and are the loveliest people ever! They are friendly and remember you if you go there often enough. He looks angry but I swear he's super nice. She's so warm and welcoming and knows my order every time I go in.Thanks.\n\nFood (and none of it is greasy!):\n+ Pork chop/cube with green sauce: INCREDIBLE. OMYGOODNESS. They're soft. They're savory. They're moist. They couldn'

## NLP Model
### Reviews Weighted for Helpfulness

In [24]:
# Create weighted DataFrame which will be used for Helpfulness model
# Binarize "Helpful" column
df_weighted = df[['useful', 'polarity', 'subjectivity', 'wc', 'text']]
df_weighted['useful'] = [1 if x >= 2 else 0 for x in df['useful']]

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [25]:
# See the percentage of "Useful" reviews
float(sum(df_weighted['useful']))/len(df_weighted['useful'])

0.2234362891619617

In [26]:
# Create our TFIDF Vectorizer which will be used to
#  rank the importance of words for our classifier
useful_vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(ngram_range=(1,2), strip_accents='unicode')
useful_vectorizer.fit(df_weighted['text'])

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents='unicode', sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [27]:
# Transform our sample of reviews into a TFIDF vector
useful_tfidf = vectorizer.transform(df_weighted['text'])

In [29]:
useful_X = df_weighted[['polarity', 'subjectivity', 'wc']]
useful_y = df_weighted['useful']

In [30]:
# Split sample into Train and Test sets for cross validation
useful_X_train, useful_X_test, useful_y_train, useful_y_test = sklearn.cross_validation.train_test_split(useful_tfidf, useful_y, test_size=0.3, stratify=useful_y)

In [31]:
# Random Forests Train Test Split
forest = RandomForestClassifier()
forest.fit(useful_X_train, useful_y_train)
for_pred = forest.predict(useful_X_test)
print "f1:", sklearn.metrics.f1_score(for_pred, useful_y_test)
print "recall:", sklearn.metrics.recall_score(for_pred, useful_y_test)
print "precision:", sklearn.metrics.precision_score(for_pred, useful_y_test)

f1: 0.0197060248748
recall: 0.448529411765
precision: 0.0100743187448


### Yeah... That doesn't work :)

## NLP Model
### Testing the Results

In [41]:
# Read the Yelp business dataset into a dictionary
# This will be used to look up Business IDs
with codecs.open('data/yelp_academic_dataset_business.json', 'r', encoding='utf-8') as f:
    b = [json.loads(line.strip()) for line in f]

In [42]:
# Create the DataFrame for training
df_bus = pd.DataFrame(b)
len(df_bus)

144072

In [55]:
df_bus

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,type
0,"227 E Baseline Rd, Ste J2","[BikeParking: True, BusinessAcceptsBitcoin: Fa...",0DI8Dt2PJp07XkVvIElIcQ,"[Tobacco Shops, Nightlife, Vape Shops, Shopping]",Tempe,"[Monday 11:0-21:0, Tuesday 11:0-21:0, Wednesda...",0,33.378214,-111.936102,Innovative Vapors,,85283,17,4.5,AZ,business
1,495 S Grand Central Pkwy,"[BusinessAcceptsBitcoin: False, BusinessAccept...",LTlCaCGZE14GuaUXUGbamg,"[Caterers, Grocery, Food, Event Planning & Ser...",Las Vegas,"[Monday 0:0-0:0, Tuesday 0:0-0:0, Wednesday 0:...",1,36.192284,-115.159272,Cut and Taste,,89106,9,5.0,NV,business
2,979 Bloor Street W,"[Alcohol: none, Ambience: {'romantic': False, ...",EDqCEAGXVGCH4FJXgqtjqg,"[Restaurants, Pizza, Chicken Wings, Italian]",Toronto,"[Monday 11:0-2:0, Tuesday 11:0-2:0, Wednesday ...",1,43.661054,-79.429089,Pizza Pizza,Dufferin Grove,M6H 1L5,7,2.5,ON,business
3,7014 Steubenville Pike,"[AcceptsInsurance: False, BusinessAcceptsCredi...",cnGIivYRLxpF7tBVR_JwWA,"[Hair Removal, Beauty & Spas, Blow Dry/Out Ser...",Oakdale,"[Tuesday 10:0-21:0, Wednesday 10:0-21:0, Thurs...",1,40.444544,-80.174540,Plush Salon and Spa,,15071,4,4.0,PA,business
4,321 Jarvis Street,"[BusinessAcceptsCreditCards: True, Restaurants...",cdk-qqJ71q6P7TJTww_DSA,"[Hotels & Travel, Event Planning & Services, H...",Toronto,None,1,43.659829,-79.375401,Comfort Inn,Downtown Core,M5B 2C2,8,3.0,ON,business
5,"30 Gibson Drive, Suite122","[BusinessAcceptsCreditCards: True, ByAppointme...",Q9rsaUiQ-A3NdEAloy0aJA,"[Nail Salons, Beauty & Spas]",Markham,"[Monday 10:30-20:0, Tuesday 10:30-20:0, Thursd...",1,43.827640,-79.342722,A Plus Nail,,L3R 2S3,3,2.5,ON,business
6,10875 N Frankloyd Wright Blvd,"[BikeParking: True, BusinessAcceptsCreditCards...",Cu4_Fheh7IrzGiK-Pc79ig,"[Baby Gear & Furniture, Shopping]",Scottsdale,"[Monday 10:0-18:0, Tuesday 10:0-18:0, Wednesda...",1,33.585271,-111.834954,Boomerang Baby,,85259,8,3.5,AZ,business
7,11072 No Frank Lloyd Wright,"[Alcohol: none, Ambience: {'romantic': False, ...",GDnbt3isfhd57T1QqU6flg,"[Tex-Mex, Mexican, Fast Food, Restaurants]",Scottsdale,"[Monday 10:0-22:0, Tuesday 10:0-22:0, Wednesda...",1,33.586710,-111.835410,Taco Bell,,85259,9,2.5,AZ,business
8,11000 North 115th Street,[BusinessAcceptsCreditCards: True],qwAHit4Tuj1zpO7CxVwOMA,"[Local Services, Self Storage]",Scottsdale,"[Monday 9:30-18:0, Tuesday 9:30-18:0, Wednesda...",1,33.586440,-111.832579,CubeSmart Self Storage,,85259,11,4.5,AZ,business
9,Hauptstr. 1,"[BikeParking: True, BusinessAcceptsCreditCards...",Nbr0kbtIrVlEcKIZoXWbSw,"[Food, Bakeries]",Stuttgart,None,1,48.729900,9.112420,Sehne Backwaren,,70563,3,3.5,BW,business
